## ETL DIM_TIEMPO

In [175]:
import  pandas as pd
from config.db_config import source, conn

### CAR

In [176]:
query_rental = "SELECT DATE(rental_date) AS Date, YEAR(rental_date) AS Year, QUARTER(rental_date) AS Quarter, MONTH(rental_date) AS Month, MONTHNAME(rental_date) AS MonthName, DAY(rental_date) AS Day FROM rental"

query_payment = "SELECT DATE(payment_date) AS Date, YEAR(payment_date) AS Year, QUARTER(payment_date) AS Quarter, MONTH(payment_date) AS Month, MONTHNAME(payment_date) AS MonthName, DAY(payment_date) AS Day FROM payment"

query_inventory = "SELECT DATE(last_update) AS Date, YEAR(last_update) AS Year, QUARTER(last_update) AS Quarter, MONTH(last_update) AS Month, MONTHNAME(last_update) AS MonthName, DAY(last_update) AS Day FROM inventory"


In [177]:
df_rental = pd.read_sql(query_rental, source)
df_payment = pd.read_sql(query_payment, source)
df_inventory = pd.read_sql(query_inventory, source)

### TRA

In [178]:
df_union = pd.concat([df_rental, df_payment, df_inventory], ignore_index=True)
df_dim_tiempo = df_union.drop_duplicates()
df_dim_tiempo = df_dim_tiempo.copy()
df_dim_tiempo['id_dim_tiempo'] = range(1, len(df_dim_tiempo) + 1)
df_dim_tiempo = df_dim_tiempo.copy()
df_colums = df_dim_tiempo[['id_dim_tiempo', 'Year', 'Month', 'Day']]

In [179]:
# Dimension date_inventory
df_date_inventory = df_colums.rename(columns={'id_dim_tiempo': 'id_date_inventory', 'Year': 'year', 'Month': 'month', 'Day': 'day'})

# Dimension date_rentals
df_date_rentals = df_colums.rename(columns={'id_dim_tiempo': 'id_date', 'Year': 'year', 'Month': 'month', 'Day': 'day'})

# Dimension date_sales
df_date_sales = df_colums.rename(columns={'id_dim_tiempo': 'id_date', 'Year': 'year', 'Month': 'month', 'Day': 'day'})

### CAR 

In [180]:
cursor = conn.cursor()
try:
    insert_inventory = """
        INSERT INTO date_inventory (id_date_inventory, year, month, day) VALUES (%s, %s, %s, %s)
        ON CONFLICT (id_date_inventory) DO UPDATE
        SET year = EXCLUDED.year, month = EXCLUDED.month, day = EXCLUDED.day
    """
    insert_rentals = """
        INSERT INTO date_rentals (id_date, year, month, day) VALUES (%s, %s, %s, %s)
        ON CONFLICT (id_date) DO UPDATE
        SET year = EXCLUDED.year, month = EXCLUDED.month, day = EXCLUDED.day
    """
    insert_sales = """
        INSERT INTO date_sales (id_date, year, month, day) VALUES (%s, %s, %s, %s)
        ON CONFLICT (id_date) DO UPDATE
        SET year = EXCLUDED.year, month = EXCLUDED.month, day = EXCLUDED.day
    """

    def insert_data_to_database(df, cursor, insert_query):
        rows_affected = 0
        for index, row in df.iterrows():
            values = tuple(row)
            cursor.execute(insert_query, values)
            rows_affected += 1
        return rows_affected

    rows_inventory = insert_data_to_database(df_date_inventory, cursor, insert_inventory)
    rows_rentals = insert_data_to_database(df_date_rentals, cursor, insert_rentals)
    rows_sales = insert_data_to_database(df_date_sales, cursor, insert_sales)

    conn.commit()
    print(f"Filas de inventario afectadas: {rows_inventory}")
    print(f"Filas de alquileres afectadas: {rows_rentals}")
    print(f"Filas de ventas afectadas: {rows_sales}")
except Exception as e:
    conn.rollback()
    print("Error durante la insercion:", e)
finally:
    cursor.close()


Filas de inventario afectadas: 42
Filas de alquileres afectadas: 42
Filas de ventas afectadas: 42
